In [1]:
    from util import *
    import pickle
    import pandas as pd
    import random
    from model import tlrnn
    from dataPrep import dataprep
    import pickle

Using TensorFlow backend.


finished


In [2]:
directory = "/home/stefan/tl_rnn/data/"

In [3]:
df = pd.read_csv(directory + "sequential_data.csv")
df = df.fillna(0)

In [4]:
df

,user_ID,sequence_ID,eventData_1,eventData_2,eventData_3
0,8509,1.0,cat_1,8,10.0
1,8509,1.0,cat_2,8,10.0
2,8509,1.0,cat_3,8,10.0
3,8509,1.0,cat_4,8,10.0
4,8509,1.0,cat_5,8,10.0
5,8509,1.0,cat_6,8,10.0
6,8509,1.0,cat_7,8,10.0
7,8509,1.0,cat_8,8,10.0
8,8509,1.0,cat_9,8,10.0
9,8509,1.0,cat_10,8,10.0


In [5]:
dataobj = dataprep(dataframe=df, id_name="user_ID", sequence_id = "sequence_ID")

DATAFRAME COLUMNS  ['user_ID', 'sequence_ID', 'eventData_1', 'eventData_2', 'eventData_3']
SEQ_LEN_MAX 100
TOP 20 Categories of all Covariates [['cat_45', 'cat_73', 'cat_44', 'cat_26', 'cat_90', 'cat_213', 'cat_160', 'cat_69', 'cat_232', 'cat_244', 'cat_752', 'cat_133', 'cat_506', 'cat_672', 'cat_425', 'cat_403', 'cat_390', 'cat_235', 'cat_83', 'cat_547', 'cat_324', 'cat_305', 'cat_47', 'cat_402', 'cat_251', 'cat_609', 'cat_217', 'cat_610', 'cat_131', 'cat_543', 'cat_470', 'cat_896', 'cat_252', 'cat_82', 'cat_122', 'cat_721', 'cat_144', 'cat_347', 'cat_143', 'cat_440', 'cat_46', 'cat_310', 'cat_1143', 'cat_240', 'cat_1139', 'cat_68', 'cat_62', 'cat_662', 'cat_112', 'cat_220', 'cat_382', 'cat_43', 'cat_1150', 'cat_262', 'cat_566', 'cat_1433', 'cat_778', 'cat_1349', 'cat_486', 'cat_89', 'cat_301', 'cat_516', 'cat_196', 'cat_86', 'cat_19', 'cat_270', 'cat_280', 'cat_1432', 'cat_253', 'cat_383', 'cat_526', 'cat_79', 'cat_409', 'cat_201', 'cat_410', 'cat_940', 'cat_1037', 'cat_293', 'cat_15

In [6]:
dataobj.preprocessor()

dataframe -> tensor ...
--------------------------------------------------
17/08/2021 19:23:17
0 of 10000 users 


/home/stefan/tl_rnn/src/dataPrep.py:139: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  np.asarray(self.vectorized_convert(df_id_t.as_matrix(columns=[col]), self.covariates_to_translate.index(col)) + 1).reshape(-1)
/home/stefan/tl_rnn/src/dataPrep.py:142: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  week_array[:, self.covariates_all.index(col)] = np.asarray(df_id_t.as_matrix(columns=[col]) + 1).reshape(-1)


--------------------------------------------------
17/08/2021 19:24:33
1000 of 10000 users 
--------------------------------------------------
17/08/2021 19:25:43
2000 of 10000 users 
--------------------------------------------------
17/08/2021 19:26:45
3000 of 10000 users 
--------------------------------------------------
17/08/2021 19:27:36
4000 of 10000 users 
--------------------------------------------------
17/08/2021 19:28:19
5000 of 10000 users 
--------------------------------------------------
17/08/2021 19:28:56
6000 of 10000 users 
--------------------------------------------------
17/08/2021 19:29:26
7000 of 10000 users 
--------------------------------------------------
17/08/2021 19:29:50
8000 of 10000 users 
--------------------------------------------------
17/08/2021 19:30:11
9000 of 10000 users 
tensor builder finished


In [17]:
dataobj.data_list[:9000]

10000

In [7]:
dataobj.build_cardinality()

{'eventData_1': 35719, 'eventData_2': 25, 'eventData_3': 32}


In [14]:
dataobj.sampler(triplets_per_user=2, users_per_file="all")

triplets per user:  2
0 of 10000 users 
1000 of 10000 users 
2000 of 10000 users 
3000 of 10000 users 
4000 of 10000 users 
5000 of 10000 users 
6000 of 10000 users 
7000 of 10000 users 
8000 of 10000 users 
9000 of 10000 users 


In [15]:
my_tlrnn = tlrnn(cells=128, dataobj=dataobj)
my_tlrnn.build()

COVARIATES:  {'eventData_1': 35719, 'eventData_2': 25, 'eventData_3': 32}
SEQUENCE LEN MAX 100
CELLS 128
PATIENCE 1
BATCH SIZE 64
BATCHES PER USER 2
OPTIMIZER <keras.optimizers.Adam object at 0x7f3fcf675c88>


In [16]:
# train generator version

alpha=1.0

run_name = "test_insta_generator"

my_tlrnn.train_generator(run_name=run_name, alpha=alpha, test_set=dataobj.triplet_tensor)

split training and validation set
length training set:  9500
length validation set:  500
compile model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 100, 1)       0                                            
___

19000/19000 [==============================] - 487s 26ms/step - loss: 0.1716 - val_loss: 0.2360
score L1:  0.9426640625
Epoch 4/12
19000/19000 [==============================] - 484s 25ms/step - loss: 0.1328 - val_loss: 0.2261
score L1:  0.95375546875
Epoch 5/12
19000/19000 [==============================] - 486s 26ms/step - loss: 0.1079 - val_loss: 0.2135
score L1:  0.960203125
Epoch 6/12
19000/19000 [==============================] - 487s 26ms/step - loss: 0.0919 - val_loss: 0.2129
score L1:  0.9657203125
Epoch 7/12
19000/19000 [==============================] - 486s 26ms/step - loss: 0.0809 - val_loss: 0.2143
score L1:  0.9692328125
Epoch 00007: early stopping
training complete!


In [17]:

max_array = np.zeros((len(dataobj.cov_number)), dtype='int32')

for user_list in dataobj.data_list:
    for seq in user_list:
        for feature in dataobj.cov_number:
            if max(seq[:, feature]) > max_array[feature]:
                max_array[feature] = max(seq[:, feature])

max_array = max_array + 1


In [20]:
COVARIATES_ALL = ["eventData_1", "eventData_2", "eventData_3"]

COVARIATES_TO_TRANSLATE = ["eventData_1"]

In [21]:
res = []
i = 0
while (i < len(COVARIATES_ALL)):
    if (COVARIATES_TO_TRANSLATE.count(COVARIATES_ALL[i]) > 0):
        res.append(i)
    i += 1

In [23]:
list1 = [1,2,3,4]
list2 = [5,6,7,8]

In [29]:
3%1

0

In [15]:
for user_list in dataobj.data_list:
    for seq in user_list:
        print(seq)
        print("---------------")
        for feature in range(3):
            print("******************")
            print(seq[:, feature])
            print("******************")
            if max(seq[:, feature]) > max_array[feature]:
                max_array[feature] = max(seq[:, feature])


[[ 1804     9    11]
 [ 2962     9    11]
 [  164     9    11]
 [ 3472     9    11]
 [ 1356     9    11]
 [ 1078     9    11]
 [29333     9    11]
 [11025     9    11]
 [15511     9    11]
 [ 3317     9    11]
 [25435     9    11]
 [25445     9    11]
 [26191     9    11]
 [ 6018     9    11]
 [26405     9    11]]
---------------
******************
[ 1804  2962   164  3472  1356  1078 29333 11025 15511  3317 25435 25445
 26191  6018 26405]
******************


NameError: name 'max_array' is not defined